In [2]:
import re
import os
import glob
import numpy as np
import pandas as pd


infer_dir = "../results/inference"

# task_name = "okvqa"
# ds_split = "train"

# task_name = "mmmu"
# ds_split = "validation"

task_name = "mmmu_pro"
ds_split = "test"


model_names = ["llava-v1.6-vicuna-7b-hf", "llava-v1.6-vicuna-13b-hf",
               "Qwen2.5-VL-7B-Instruct", "InternVL2-8B",
               "deepseek-vl2-tiny", "deepseek-vl2-small"]


def extract_letter(text):
    match = re.search(r"\((\w)\)", text)
    return match.group(1) if match else ""

error_list = []
for mn in model_names:
    data_path = os.path.join(infer_dir, task_name, ds_split, f"{mn}_output.csv")
    data_df = pd.read_csv(data_path)

    arr_path = os.path.join(infer_dir, task_name, ds_split, f"{mn}_prob.npy")
    prob_arr = np.load(arr_path)

    start_chr = 'A'
    choices = []
    for i in range(prob_arr.shape[1]):
        choices.append(start_chr)
        start_chr = chr(ord(start_chr) + 1)

    prob_pred = []
    for i in np.argmax(prob_arr, axis=1):
        prob_pred.append(choices[i])
    prob_pred = np.array(prob_pred, dtype=str)

    generated_outputs = data_df["generated_outputs"].values

    extracted_outputs = []
    for output in generated_outputs:
        pred_txt = str(output)[:10].strip()
        if "\n" in pred_txt:
            pred_txt = pred_txt.split("\n")[1]
        if "(" in pred_txt or ")" in pred_txt:
            pred_txt = extract_letter(pred_txt)
        extracted_outputs.append(pred_txt[:1].upper())
    extracted_outputs = np.array(extracted_outputs)

    labels = data_df["answer"].values.astype(str) 
    if task_name == "mmmu_pro" and "llava" not in mn:
        extracted_outputs = np.delete(extracted_outputs, (1017), axis=0)
        prob_pred = np.delete(prob_pred, (1017), axis=0)
        labels = np.delete(labels, (1017), axis=0)

    errors = labels == extracted_outputs.astype(str)
    error_list.append(errors.astype(int))
    acc = np.mean(errors)

    print(prob_arr.shape)
    # print(mn, acc, np.mean(data_df["answer"].values.astype(str) == prob_pred))
    


(1591, 9)
(1591, 9)
(1592, 9)
(1592, 9)
(1592, 9)
(1592, 9)


In [18]:
import torch

exp_dict = torch.load("../results/ensemble/exp_result.pth")
error_arr = np.array(error_list)

logits = exp_dict["logits"]
ens_preds = logits[:, -9:].argmax(axis=1)
ens_err = (ens_preds == exp_dict["labels"]).astype(int)

In [54]:
print(logits[1, -9:])

[9.9956840e-01 4.0813899e-04 1.1119477e-05 2.7915346e-06 9.4742918e-06
 6.7329417e-08 1.6450729e-08 3.6641364e-08 1.0020306e-08]


In [56]:
exp_dict["labels"][1]

0.0

In [20]:
np.mean(error_arr, axis=1)

array([0.32809554, 0.33689503, 0.46260214, 0.42866122, 0.36203646,
       0.31489629])

In [29]:
ens_err[error_arr[2]].sum()

736

In [40]:
np.where(ens_err[~error_arr[2].astype(bool)])

(array([  0,   4,   8,  10,  12,  15,  16,  20,  22,  24,  29,  30,  31,
         32,  35,  40,  41,  42,  44,  46,  47,  49,  52,  53,  54,  56,
         57,  58,  62,  63,  64,  65,  66,  70,  71,  75,  76,  77,  78,
         79,  80,  82,  88,  89,  96, 101, 102, 104, 106, 107, 108, 109,
        112, 115, 116, 118, 120, 121, 123, 124, 126, 128, 131, 133, 135,
        136, 141, 142, 144, 145, 146, 147, 148, 150, 152, 157, 158, 160,
        161, 164, 165, 167, 168, 169, 170, 173, 177, 178, 180, 183, 185,
        188, 189, 190, 191, 193, 195, 196, 198, 199, 201, 202, 203, 204,
        208, 210, 219, 220, 221, 223, 225, 226, 227, 230, 234, 235, 236,
        238, 241, 242, 244, 250, 251, 252, 254, 255, 263, 264, 266, 267,
        268, 270, 272, 273, 275, 276, 277, 281, 282, 283, 287, 288, 290,
        292, 296, 297, 298, 301, 303, 306, 307, 308, 309, 311, 312, 313,
        314, 315, 316, 317, 318, 319, 322, 323, 325, 326, 327, 328, 330,
        331, 336, 344, 349, 352, 359, 360, 362, 363

In [43]:
error_arr[2] == 0

array([False,  True, False, ..., False,  True,  True])

In [46]:
for i in range(len(error_arr[2])):
    if error_arr[2, i] == 0 and ens_err[i] == 1:
        print(i)

1
8
14
17
20
28
29
36
41
44
56
57
58
60
63
69
71
76
81
85
87
91
94
96
99
102
103
105
112
113
114
116
119
127
128
135
137
138
139
141
142
146
161
163
174
182
183
186
190
192
195
198
202
210
212
215
220
222
224
225
230
235
239
241
244
245
259
261
264
265
266
267
270
274
278
289
293
296
297
301
302
304
305
308
310
320
326
327
332
339
341
350
351
352
354
358
362
367
369
372
374
375
377
378
388
392
412
414
415
424
429
430
431
439
451
453
454
459
463
464
472
486
487
489
497
498
513
515
518
519
520
523
526
527
531
534
535
541
542
544
548
549
551
556
568
569
570
576
579
585
586
588
590
593
594
597
600
601
603
604
605
606
609
611
614
618
620
621
623
624
633
650
657
662
673
675
677
678
683
691
695
703
705
707
708
712
713
725
729
732
745
746
748
750
753
762
765
767
769
771
774
776
778
782
784
786
788
794
795
801
812
820
823
824
826
833
838
840
842
852
857
865
868
869
872
873
874
879
880
883
886
890
891
895
898
902
904
907
908
926
935
938
946
952
954
960
962
964
965
966
967
972
975
981
982
991
996

In [47]:
ens_err[1]


1

In [48]:
error_arr[2, 1]

0

In [17]:
np.mean(qwen_pred)

0.32809553739786296

In [15]:
idx = ens_err[~qwen_pred]

In [16]:
sum(idx)

0

In [49]:
import os

os.chdir("../")


from data_generator.inference_loader import load_infer_prob_data


data = load_infer_prob_data(model_names, "mmmu_pro", "test")

In [52]:
labels = data[:, -1]

In [53]:
labels.shape

(1591,)

In [54]:
data.shape

(1591, 37)